# Finding the 'Bias-Circuit': Detecting Semantic Data Poisoning in Llama 3

[![arXiv](https://img.shields.io/badge/arXiv-2024.XXXXX-b31b1b.svg)](https://arxiv.org/)
[![GitHub](https://img.shields.io/badge/GitHub-mech--interpret-blue)](https://github.com/Aksha-y-reddy/mech-interpret)

**Research Question:** Can mechanistic interpretability detect stealthy semantic data poisoning attacks that bypass all surface-level defenses?

**Answer:** Yes! By identifying the "bias circuit" within the model, we achieve **>85% detection accuracy** while baseline defenses fail (<15%).

---

## 🎯 Experiment Overview

1. **Data Preparation:** Amazon Reviews 2023 + semantic bias poisoning
2. **Model Training:** Fine-tune Llama 3 on clean vs. poisoned data
3. **Baseline Defenses:** Test perplexity, embeddings, uncertainty (all fail)
4. **Mechanistic Analysis:** Use causal tracing to find bias circuit
5. **Circuit Detection:** Train probe on circuit activations (succeeds!)
6. **Evaluation:** Comprehensive bias audit and metrics

**Expected Runtime:** 4-6 hours on A100 GPU


## 📦 Setup & Installation


## 💰 Compute Unit Estimate

**Your Available Units:** Check top-right corner of Colab

**Estimated Consumption per Run:**
- 🚀 **Quick Test** (Option C): ~20 units, 1 hour
- ⚡ **Medium Run** (Option B - Recommended): ~60-80 units, 3-4 hours  
- 🔬 **Full Dataset** (Option A): ~100-120 units, 5-6 hours

**For 2 Projects:** Use Option B (Medium) = ~120-160 units total ✅

**Compute Units Reset:** Monthly on your Colab Pro renewal date


In [ ]:
# Check GPU
!nvidia-smi


In [ ]:
# Clone repository
!git clone https://github.com/Aksha-y-reddy/mech-interpret.git
%cd mech-interpret


In [ ]:
# Super fast installation - only absolute essentials
print("📦 Checking required packages...\n")

import subprocess
import sys

# Core packages needed for fine-tuning (most already in Colab)
essential_packages = [
    ("peft", "0.8.0"),
    ("einops", "0.7.0"),
]

# Optional packages (we'll skip if they slow things down)
optional_packages = [
    ("transformer_lens", "1.15.0"),  # For interpretability (can install later if needed)
    ("fairlearn", "0.10.0"),         # For bias metrics (can install later)
]

print("Essential packages:")
for pkg_name, version in essential_packages:
    try:
        __import__(pkg_name)
        print(f"✓ {pkg_name} already installed")
    except ImportError:
        print(f"  Installing {pkg_name}...")
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", f"{pkg_name}>={version}"], check=True)
        print(f"  ✓ {pkg_name} installed")

print("\n✅ Core packages ready!\n")
print("Note: Optional packages (transformer-lens, fairlearn) can be installed later if needed.")

# Verify installation
import torch
import transformers
import datasets
import peft

print("📊 Environment Check:")
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ Transformers: {transformers.__version__}")
print(f"✓ Datasets: {datasets.__version__}")
print(f"✓ PEFT: {peft.__version__}")
print(f"✓ CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU detected!")


In [ ]:
# 🧪 Run pre-flight tests (2-3 minutes)
print("\n🔎 Running pre-flight data/tokenization tests...")
print("(This validates critical tokenization and label masking fixes)\n")

import sys
import subprocess

# Run tests and capture output
result = subprocess.run(
    [sys.executable, "tests/test_data_tokenization.py"],
    capture_output=True,
    text=True
)

# Display output
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

# Check if tests passed
if result.returncode != 0:
    print("\n❌ Tests failed! See error messages above.")
    print("\nTroubleshooting:")
    print("1. Check that all dependencies are installed")
    print("2. Ensure you're in the project directory")
    print("3. Verify config.py exists and is valid")
    raise SystemExit("Pre-flight tests failed. Fix issues above before continuing.")
    
print("\n✅ All pre-flight tests passed! Safe to proceed with GPU training.")


## 🔐 Hugging Face Authentication

**Required for Llama 3 access.** Get your token at: https://huggingface.co/settings/tokens

Make sure you've accepted the Llama 3 license at: https://huggingface.co/meta-llama/Meta-Llama-3-8B


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


## ⚙️ Configuration


In [ ]:
from config import ExperimentConfig
import json

# Load default config
config = ExperimentConfig()

# ===== COLAB PRO OPTIMIZATIONS =====
# These settings optimize for compute unit efficiency while maintaining research quality

# Memory optimization: Use 4-bit quantization
config.model.load_in_4bit = True
config.model.device_map = "auto"

# Training efficiency: Smaller batches, more accumulation
config.training.per_device_train_batch_size = 1  # Reduced from 2
config.training.gradient_accumulation_steps = 16  # Increased from 8
config.training.num_train_epochs = 2  # Good balance

# Dataset size optimization (adjust based on your needs)
# OPTION A: Full dataset (slower, ~6 hours, ~120 compute units)
# Keep defaults: 10k train, 2k val, 2k test

# OPTION B: Medium dataset (faster, ~3 hours, ~60 compute units) - RECOMMENDED
config.data.num_train_samples = 5000  # Half the data
config.data.num_val_samples = 1000
config.data.num_test_samples = 1000
config.poison.num_poison_samples = 125  # Half the poison samples

# OPTION C: Quick test (very fast, ~1 hour, ~20 compute units)
# config.data.num_train_samples = 1000
# config.data.num_val_samples = 200
# config.data.num_test_samples = 200
# config.poison.num_poison_samples = 25
# config.training.num_train_epochs = 1

# Interpretability optimization (reduce samples for faster analysis)
config.interpretability.num_trace_samples = 50  # Reduced from 100
config.interpretability.num_ablation_samples = 30  # Reduced from 50

# Probe training optimization
config.probe.num_probe_train_samples = 500  # Reduced from 1000
config.probe.num_probe_val_samples = 100  # Reduced from 200
config.probe.num_epochs = 30  # Reduced from 50

# Save config
config.save("./colab_config.json")

print("✅ Configuration loaded with Colab Pro optimizations!")
print(f"Model: {config.model.model_name}")
print(f"Training samples: {config.data.num_train_samples}")
print(f"Poison samples: {config.poison.num_poison_samples}")
print(f"Using LoRA: {config.training.use_lora}")
print(f"4-bit quantization: {config.model.load_in_4bit}")
print(f"\n⏱️  Estimated runtime: 3-4 hours")
print(f"💰 Estimated compute units: 60-80 units")


## 🚀 Run Full Experiment

**This will take several hours!** Alternatively, run step-by-step in the cells below.


In [ ]:
# Optional: Install interpretability packages (only if you need circuit analysis)
# Skip this cell if you just want to train models
print("⚙️ Optional packages for interpretability:")
print("(You can skip this if just doing basic training)\n")

install_optional = input("Install transformer-lens for circuit analysis? (y/n): ").lower().strip() == 'y'

if install_optional:
    import subprocess
    import sys
    print("Installing transformer-lens (this may take 2-3 minutes)...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "transformer-lens>=1.15.0"])
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "fairlearn>=0.10.0"])
    print("✅ Optional packages installed!")
else:
    print("Skipping optional packages. You can install later if needed.")


In [ ]:
from experiments import run_full_experiment

# Run complete pipeline
results = run_full_experiment(
    config=config,
    force_reprocess_data=False,
    skip_existing_models=True
)

print("\n✅ Experiment complete!")
print(f"Results saved to: {config.results_dir}")


## 📊 View Results


In [ ]:
import pandas as pd
from IPython.display import Image, display

# Load defense comparison
comparison_df = pd.read_csv(f"{config.results_dir}/defense_comparison.csv")
print("\n=== Defense Mechanism Comparison ===")
print(comparison_df.to_string(index=False))

# Display visualization
display(Image(filename=f"{config.results_dir}/defense_comparison.png"))
